In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime

In [5]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install catboost

Note: you may need to restart the kernel to use updated packages.


In [11]:
import seaborn as sns
from catboost import CatBoostClassifier
import catboost as cb

In [12]:
train_data = pd.read_csv("D:/Misc/Forecast/amexpert-2019/train.csv")
cust_demo_data = pd.read_csv("D:/Misc/Forecast/amexpert-2019/customer_demographics.csv")
campaign_data = pd.read_csv("D:/Misc/Forecast/amexpert-2019/campaign_data.csv")
coupon_item_mapping = pd.read_csv("D:/Misc/Forecast/amexpert-2019/coupon_item_mapping.csv")
customer_transaction_data = pd.read_csv("D:/Misc/Forecast/amexpert-2019/customer_transaction_data.csv")
item_data = pd.read_csv("D:/Misc/Forecast/amexpert-2019/item_data.csv")

In [13]:
cust_demo_data["no_of_children"].describe()

count     222
unique      3
top         1
freq      107
Name: no_of_children, dtype: object

In [14]:
cust_demo_data["no_of_children"].fillna(0, inplace=True)

In [15]:
cust_demo_data["marital_status"] = np.where(cust_demo_data["marital_status"].isna(), np.where(cust_demo_data["no_of_children"]==0,"Single","Married"),cust_demo_data["marital_status"])

In [16]:
cust_demo_data["marital_status"].count()

760

In [17]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78369 entries, 0 to 78368
Data columns (total 5 columns):
id                   78369 non-null int64
campaign_id          78369 non-null int64
coupon_id            78369 non-null int64
customer_id          78369 non-null int64
redemption_status    78369 non-null int64
dtypes: int64(5)
memory usage: 3.0 MB


In [18]:
train_data.head()

,id,campaign_id,coupon_id,customer_id,redemption_status
0,1,13,27,1053,0
1,2,13,116,48,0
2,6,9,635,205,0
3,7,13,644,1050,0
4,9,8,1017,1489,0


In [19]:
campaign_data.head()

,campaign_id,campaign_type,start_date,end_date
0,24,Y,21/10/13,20/12/13
1,25,Y,21/10/13,22/11/13
2,20,Y,07/09/13,16/11/13
3,23,Y,08/10/13,15/11/13
4,21,Y,16/09/13,18/10/13


In [20]:
train_data_merge = pd.merge(train_data, campaign_data, how="inner", on="campaign_id")

In [21]:
coupon_item_mapping.head()

,coupon_id,item_id
0,105,37
1,107,75
2,494,76
3,522,77
4,518,77


In [22]:
item_data.head()

,item_id,brand,brand_type,category
0,1,1,Established,Grocery
1,2,1,Established,Miscellaneous
2,3,56,Local,Bakery
3,4,56,Local,Grocery
4,5,56,Local,Grocery


In [23]:
cust_demo_data.head()

,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,1,70+,Married,0,2,0,4
1,6,46-55,Married,0,2,0,5
2,7,26-35,Married,0,3,1,3
3,8,26-35,Married,0,4,2,6
4,10,46-55,Single,0,1,0,5


In [24]:
train_data_merge.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,start_date,end_date
0,1,13,27,1053,0,X,19/05/13,05/07/13
1,2,13,116,48,0,X,19/05/13,05/07/13
2,7,13,644,1050,0,X,19/05/13,05/07/13
3,21,13,1028,89,0,X,19/05/13,05/07/13
4,23,13,517,1067,0,X,19/05/13,05/07/13


In [25]:
train_data_merge = pd.merge(train_data_merge, coupon_item_mapping, how="inner", on="coupon_id")

In [26]:
train_data_merge = pd.merge(train_data_merge, item_data, how="inner", on="item_id")

In [27]:
train_data_merge = pd.merge(train_data_merge, cust_demo_data, how="inner", on="customer_id")

In [28]:
train_data_merge.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,start_date,end_date,item_id,brand,brand_type,category,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,1,13,27,1053,0,X,19/05/13,05/07/13,24775,1636,Established,Grocery,46-55,Single,0,1,0,5
1,68170,13,22,1053,0,X,19/05/13,05/07/13,24775,1636,Established,Grocery,46-55,Single,0,1,0,5
2,1,13,27,1053,0,X,19/05/13,05/07/13,14958,1636,Established,Grocery,46-55,Single,0,1,0,5
3,68170,13,22,1053,0,X,19/05/13,05/07/13,14958,1636,Established,Grocery,46-55,Single,0,1,0,5
4,1,13,27,1053,0,X,19/05/13,05/07/13,40431,1636,Established,Grocery,46-55,Single,0,1,0,5


In [29]:
pd.unique(train_data_merge[['id', 'campaign_id', 'coupon_id', 'customer_id', 'redemption_status', 'item_id']].values.ravel())

array([     1,     13,     27, ...,  80567, 117816,  18819], dtype=int64)

In [30]:
train_data_merge.count()

id                   3807856
campaign_id          3807856
coupon_id            3807856
customer_id          3807856
redemption_status    3807856
campaign_type        3807856
start_date           3807856
end_date             3807856
item_id              3807856
brand                3807856
brand_type           3807856
category             3807856
age_range            3807856
marital_status       3807856
rented               3807856
family_size          3807856
no_of_children       3807856
income_bracket       3807856
dtype: int64

In [31]:
train_data_merge.drop_duplicates(keep="first", inplace=True)

In [32]:
train_data_merge.count()

id                   3807856
campaign_id          3807856
coupon_id            3807856
customer_id          3807856
redemption_status    3807856
campaign_type        3807856
start_date           3807856
end_date             3807856
item_id              3807856
brand                3807856
brand_type           3807856
category             3807856
age_range            3807856
marital_status       3807856
rented               3807856
family_size          3807856
no_of_children       3807856
income_bracket       3807856
dtype: int64

In [33]:
customer_transaction_data.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0
2,2012-01-02,1501,31962,1,106.50,-14.25,0.0
3,2012-01-02,1501,33647,1,67.32,0.00,0.0
4,2012-01-02,1501,48199,1,71.24,-28.14,0.0


In [34]:
train_data_merge = pd.merge(train_data_merge, customer_transaction_data, how="left", on=['customer_id', 'item_id'])

In [35]:
train_data_merge.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,start_date,end_date,item_id,brand,...,marital_status,rented,family_size,no_of_children,income_bracket,date,quantity,selling_price,other_discount,coupon_discount
0,1,13,27,1053,0,X,19/05/13,05/07/13,24775,1636,...,Single,0,1,0,5,NaN,NaN,NaN,NaN,NaN
1,68170,13,22,1053,0,X,19/05/13,05/07/13,24775,1636,...,Single,0,1,0,5,NaN,NaN,NaN,NaN,NaN
2,1,13,27,1053,0,X,19/05/13,05/07/13,14958,1636,...,Single,0,1,0,5,NaN,NaN,NaN,NaN,NaN
3,68170,13,22,1053,0,X,19/05/13,05/07/13,14958,1636,...,Single,0,1,0,5,NaN,NaN,NaN,NaN,NaN
4,1,13,27,1053,0,X,19/05/13,05/07/13,40431,1636,...,Single,0,1,0,5,NaN,NaN,NaN,NaN,NaN


In [36]:
train_data_merge.drop_duplicates(keep="first", inplace=True)

In [37]:
train_data_merge['quantity'].fillna(0, inplace=True)

In [38]:
train_data_merge['other_discount'].fillna(0, inplace=True)

In [39]:
train_data_merge['coupon_discount'].fillna(0, inplace=True)

In [40]:
train_data_merge.columns

Index(['id', 'campaign_id', 'coupon_id', 'customer_id', 'redemption_status',
       'campaign_type', 'start_date', 'end_date', 'item_id', 'brand',
       'brand_type', 'category', 'age_range', 'marital_status', 'rented',
       'family_size', 'no_of_children', 'income_bracket', 'date', 'quantity',
       'selling_price', 'other_discount', 'coupon_discount'],
      dtype='object')

In [41]:
columns = ['campaign_type','brand_type', 'category', 'quantity','other_discount', 'coupon_discount','marital_status',
           'rented','income_bracket','age_range','brand']
x_data = train_data_merge[columns]
y_data = train_data_merge['redemption_status']

In [42]:
x_data.isnull().sum()

campaign_type      0
brand_type         0
category           0
quantity           0
other_discount     0
coupon_discount    0
marital_status     0
rented             0
income_bracket     0
age_range          0
brand              0
dtype: int64

In [43]:
from catboost import CatBoostClassifier
categorical_var = np.where(x_data.dtypes != np.float)[0]
print(categorical_var)

[ 0  1  2  6  7  8  9 10]


In [44]:
cat_model = CatBoostClassifier(iterations=50, learning_rate=0.7)
cat_model.fit(x_data, y_data, cat_features=categorical_var, plot=False)

0:	learn: 0.3353197	total: 1.45s	remaining: 1m 10s
1:	learn: 0.2456362	total: 2.58s	remaining: 1m 2s
2:	learn: 0.2116840	total: 3.42s	remaining: 53.7s
3:	learn: 0.1975750	total: 4.38s	remaining: 50.4s
4:	learn: 0.1908802	total: 5.39s	remaining: 48.5s
5:	learn: 0.1824292	total: 6.54s	remaining: 48s
6:	learn: 0.1777466	total: 7.58s	remaining: 46.5s
7:	learn: 0.1715932	total: 8.59s	remaining: 45.1s
8:	learn: 0.1688158	total: 9.62s	remaining: 43.8s
9:	learn: 0.1652166	total: 10.4s	remaining: 41.8s
10:	learn: 0.1643858	total: 11.2s	remaining: 39.8s
11:	learn: 0.1632214	total: 12s	remaining: 38s
12:	learn: 0.1627313	total: 12.8s	remaining: 36.4s
13:	learn: 0.1609755	total: 13.6s	remaining: 34.9s
14:	learn: 0.1594347	total: 14.4s	remaining: 33.6s
15:	learn: 0.1589771	total: 15.2s	remaining: 32.3s
16:	learn: 0.1586836	total: 16s	remaining: 31s
17:	learn: 0.1583957	total: 16.7s	remaining: 29.8s
18:	learn: 0.1580121	total: 17.5s	remaining: 28.6s
19:	learn: 0.1567318	total: 18.4s	remaining: 27.6s

In [45]:
y_pred = cat_model.predict(x_data)
from sklearn.metrics import confusion_matrix, accuracy_score
confusion_matrix = confusion_matrix(y_data, y_pred)
print(confusion_matrix)
print("Accuracy:", accuracy_score(y_data, y_pred))

[[3561908   18441]
 [ 219003   53958]]
Accuracy: 0.9383792116388248
